In [18]:
import numpy as np
import pandas as pd
# 导入当前项目路径以便找到自己写的包
import sys
sys.path.append("C:Users\perma\PycharmProjects\paper")
from algorithm.km_smote import Over_Sample
from algorithm.under_sample import Under_Sample

In [19]:
def turn_name_to_num(name):
    names = ["ME2", "ME1", "EXC", "VAC", "POX", "ERL"]            
    if name in names:
        return 1.0
    else:
        return 0.0
# 处理数据的总函数
def process(data_path):
    names = ["Sequence Name", "mcg", "gvh", "alm", "mit", "erl", "pox", "vac", "nuc", "class"]
    data = pd.read_table(filepath_or_buffer=data_path, 
                         header=None, 
                         index_col=None, 
                         names=names, 
                         sep="\s+")
    data["class"] = data["class"].apply(turn_name_to_num)
    columns = ["mcg", "gvh", "alm", "mit", "erl", "pox", "vac", "nuc"]
    data_ = data[columns]
    label = data["class"]
    return data_, label
root_path = "C:\\Users\\perma\\PycharmProjects\\paper"
data_path = root_path + "\\data\\yeast.data"
data, label = process(data_path)
print(data.head(3))
print("data shape:", data.shape, "label shape", label.shape)
print(label.head(3))
c1 = 0 
c0 = 0
for c in label:
    if c == 1.0:
        c1 += 1
    else:
        c0 += 1
print("1_count", c1, "0_count", c0)

    mcg   gvh   alm   mit  erl  pox   vac   nuc
0  0.58  0.61  0.47  0.13  0.5  0.0  0.48  0.22
1  0.43  0.67  0.48  0.27  0.5  0.0  0.53  0.22
2  0.64  0.62  0.49  0.15  0.5  0.0  0.53  0.22
data shape: (1484, 8) label shape (1484,)
0    0.0
1    0.0
2    0.0
Name: class, dtype: float64
1_count 185 0_count 1299


D:\software\anaconda\exe\envs\forpaper\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, make_scorer, f1_score
from sklearn.model_selection import train_test_split

In [21]:
# knn = KNeighborsClassifier()
# dc = DecisionTreeClassifier()
# svm = SVC()
# lg = LogisticRegression()
# nb = GaussianNB()
# algorithm = {"knn": knn, "tree": dc, "svm": svm, "lgs": lg, "nb": nb}
# alg_params = {"knn": {"n_neighbors": 7}, 
#               "tree": {}, 
#               "svm": {}, 
#               "lgs": {}, 
#               "nb": {}}

In [22]:
# 这种实验有很多预测对的少数类样本是属于生成的样本，并不能代表对实际少数类的真实预测情况
# original的可以代表对真实的预测情况
# 所以决定用合成少数类来训练样本，把真实的样本留在预测
# 或者预留一部分作为测试集
# 设实验次数为n
# for i in range(n):
#    将数据集分为训练集和测试集，每个类样本的在训练集和测试集比例为3：1
#    在训练集上合成数据训练模型
#    在测试集上测试结果
#取n次实验结果的平均值为最终成绩

# 我自己的算法不一样需要改一下



In [23]:
# n 为实验次数
n = 100
# 集成的模型个数
ensemble = 3
keys = ["knn", "tree", "svm", "lgs", "nb"]
constructor = {"knn": KNeighborsClassifier, "tree": DecisionTreeClassifier, "svm": SVC, 
               "lgs": LogisticRegression, "nb": GaussianNB}
algorithm_args = {"knn": {"n_neighbors": 7}, "tree": {}, "svm": {}, "lgs": {}, "nb": {}}
algorithm = dict([(key, list()) for key in keys])
for key in keys:
    for i in range(ensemble):
        construc = constructor[key]
        args = algorithm_args[key]
        alg = construc(**args)
        algorithm[key].append(alg)



In [38]:
# 定义gmean函数
def g_mean(ground_truth, prediction):
    matrix = confusion_matrix(y_true=ground_truth, y_pred=prediction)
    tpr = matrix[1, 1] / (matrix[1, 1] + matrix[1, 0])
    tpn = matrix[0, 0] / (matrix[0, 0] + matrix[0, 1])
    return np.sqrt(tpr * tpn)
# 存储每次结果
result = dict([(key, {"gmean": list(), "fscore": list()}) for key in algorithm.keys()])
major = []
minor = []
major_label = []
minor_label = []
# 先将样本分为多数类和少数类
for index in range(len(label)):
    if label[index] == 1.0:
        minor.append(data.iloc[index, :].to_list())
        minor_label.append(1.0)
    else:
        major.append(data.iloc[index, :].to_list())
        major_label.append(0.0)
for i in range(n):
    if i % 25 == 0:
        print("round: ", i)
    # 先将数据集分为训练和测试集
    major_train, major_test, major_label_train, major_label_test = train_test_split(major, major_label, 
                                                                                    test_size=0.25, shuffle=True)
    minor_train, minor_test, minor_label_train, minor_label_test = train_test_split(minor, minor_label, 
                                                                                    test_size=0.25, shuffle=True)
    train = np.concatenate((major_train, minor_train), axis=0)
    train_label = np.concatenate((major_label_train, minor_label_train), axis=0)
    test = np.concatenate((major_test, minor_test))
    test_label = np.concatenate((major_label_test, minor_label_test), axis=0)
    predictions = dict((key, list()) for key in keys)
    for j in range(ensemble):
        kmeans_arg = {"n_clusters": 10}
        over_sampler = Over_Sample(data=train, label=train_label, n=5, categorical_features=[], **kmeans_arg)
        syntheticed_samples = over_sampler.do_synthetic()
        syntheticed_labels = len(syntheticed_samples) * [1.0]
        under_sampler = Under_Sample(major=major_train, major_label=major_label_train, synthetics=syntheticed_samples,
                                     synthetics_label=syntheticed_labels, categorical_features=[], rate=0.0,
                                     **kmeans_arg)
        under_samples = under_sampler.do_undersample()
        under_labels = len(under_samples) * [0.0]
        over_under_samples = np.concatenate((syntheticed_samples, under_samples), axis=0)
        over_under_labels = np.concatenate((syntheticed_labels, under_labels), axis=0)
        for key in keys:
            for alg in algorithm[key]:
                alg.fit(X=over_under_samples, y=over_under_labels)
                prediction = alg.predict(X=test)
                predictions[key].append(prediction)
    for key in keys:
        prediction = predictions[key]
        pre = []
        for z, x, c in zip(prediction[0], prediction[1], prediction[2]):
            if z == x:
                pre.append(z)
            else:
                pre.append(c)
        fscore = f1_score(y_true=test_label, y_pred=pre) / 2
        gmean = g_mean(ground_truth=test_label, prediction=pre)
        result[key]["fscore"].append(fscore)
        result[key]["gmean"].append(gmean)
new_line = 0       
for alg_key in result.keys():
    new_line += 1
    alg = result[alg_key]
    for score_key in alg.keys():
        mean_score = np.mean(alg[score_key])
        print(alg_key, " : ", score_key, " : ", mean_score, end="-----------------")
    print("\n")


round:  0


round:  25


round:  50


round:  75


knn  :  gmean  :  0.0-----------------knn  :  fscore  :  0.11217183770883052-----------------

tree  :  gmean  :  0.30680159814357977-----------------tree  :  fscore  :  0.11944375881250113-----------------

svm  :  gmean  :  0.0-----------------svm  :  fscore  :  0.11217183770883052-----------------

lgs  :  gmean  :  0.0-----------------lgs  :  fscore  :  0.11217183770883052-----------------

nb  :  gmean  :  0.3592642997462769-----------------nb  :  fscore  :  0.1176470119437251-----------------



In [ ]:
# ensem = 3
#kmeans_arg = {"n_clusters": 10}
# rate =0.5
# over_sampe n = 6
# knn  :  gmean  :  0.8161940257044472-----------------knn  :  fscore  :  0.29362386974321875-----------------
# 
# tree  :  gmean  :  0.6770887577269807-----------------tree  :  fscore  :  0.2337023040470957-----------------
# 
# svm  :  gmean  :  0.8419578240385807-----------------svm  :  fscore  :  0.34976042979187627-----------------
# 
# lgs  :  gmean  :  0.8177024627267186-----------------lgs  :  fscore  :  0.3499989420274296-----------------
# 
# nb  :  gmean  :  0.801173567270654-----------------nb  :  fscore  :  0.3248447740605052-----------------


In [ ]:
# ensemble = 3
# kmeans_arg = {"n_clusters": 20}
# over_sample n = 6
# under_sample rate = 0.5
# knn  :  gmean  :  0.8193007161501623-----------------knn  :  fscore  :  0.2950527637111551-----------------
# 
# tree  :  gmean  :  0.7054201631703559-----------------tree  :  fscore  :  0.2461424484415236-----------------
# 
# svm  :  gmean  :  0.8478393353679243-----------------svm  :  fscore  :  0.3544252798104335-----------------
# 
# lgs  :  gmean  :  0.8312384224759848-----------------lgs  :  fscore  :  0.3511362949594299-----------------
# 
# nb  :  gmean  :  0.7928233472575573-----------------nb  :  fscore  :  0.3317873151455374-----------------